In [3]:
from main import Group, grigora2, Worker, expert_parallel_group_objective_function, all_reduce_function, gamma_function
import numpy as np

In [71]:
adjacency = np.array([[(None, None), (0, 1), (0, 2), (None, None), (None, None), (None, None)],
                          [(0, 1), (None, None), (None, None), (0, 3), (None, None), (None, None)],
                          [(0, 2), (None, None), (None, None), (None, None), (0, 5), (None, None)],
                          [(None, None), (0, 3), (None, None), (None, None), (None, None), (0, 2)],
                          [(None, None), (None, None), (0, 5), (None, None), (None, None), (0, 2)],
                          [(None, None), (None, None), (None, None), (0, 2), (0, 2), (None, None)]])

w = []
for ii in range(adjacency.shape[0]):
    w.append(Worker(ii, 1, 12))
n_exp = 12
exp = []
for ii in range(n_exp):
    exp.append(1)
p2p_buf = 1
p2p_fr = 4
all_r_buf = 1
gamma_arguments = {Group.NUM_LAYERS: 4,
                   Group.GLOBAL_BATCH_SIZE: 4,
                   Group.MINI_BATCH_SIZE: 2,
                   Group.MOE_FREQUENCY: 2,
                   Group.RECOMPUTATION_AMOUNT: 1}

In [72]:
print(grigora2(a=adjacency,
                   obj=expert_parallel_group_objective_function,
                   all_reduce_func=all_reduce_function,
                   gamma=gamma_function,
                   p2p_buffer_size=p2p_buf,
                   p2p_freq=p2p_fr,
                   all_reduce_buffer_size=all_r_buf,
                   workers=w,
                   expert_workload=exp,
                   gamma_args=gamma_arguments).subsets())

[{0, 1}, {2}, {3}, {4}, {5}]


In [78]:
from sklearn.cluster import SpectralClustering
adjacency = np.array([[0, 2, 2, 0, 0, 0],
                          [1, 0, 0, 3, 0, 0],
                          [2, 0, 0, 0, 5, 0],
                          [0, 3, 0, 0, 0, 2],
                          [0, 0, 5, 0, 0, 2],
                          [0, 0, 0, 2, 2, 0]])
delta = 1
k_adj = np.exp(- adjacency ** 2 / (2. * delta ** 2))
sc = SpectralClustering(5, n_neighbors=2, affinity='precomputed_nearest_neighbors', assign_labels='cluster_qr')
print(sc.fit_predict(k_adj))
#verbose=True, eigen_solver='amg'

[2 1 0 3 0 4]
